# Finetuning MoonShine in PyTorch

## Data
Using the Tensorflow mini speech command dataset to get up and running. See https://www.tensorflow.org/tutorials/audio/simple_audio for sources

In [4]:
from download_mini_speech_commands import downloadMiniSpeech

downloadMiniSpeech()

Download complete.
Extracting dataset...
Extraction complete.
